In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import os
import torch
from torch.utils.data import DataLoader

In [ ]:
directory_acc = 'data/starman_images2D_noacc'
abnormal_subjects = 'data/extreme_values_images'

df_acc = pd.read_csv(os.path.join(directory_acc, 'starmen_noacc.csv'))
df_abnormal_subjects = pd.read_csv(os.path.join(abnormal_subjects, 'extreme_values_acc.csv'))

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
from utils import id_based_train_test_split

train_df, val_df, test_df = id_based_train_test_split(df_acc, val_size=0.1, test_size=0.1, random_state=42)

In [ ]:
from datasets2D import Dataset2D, DatasetLongitudinal2D, ID_Dataset2D

train_dataset = DatasetLongitudinal2D(train_df)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_dataset = DatasetLongitudinal2D(val_df)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

In [ ]:
# TRAIN REGULAR VAE

from VAE2D import VAE2D
from training_bVAE2D import train_bVAE

latent_representation_size = 4
model = VAE2D(latent_representation_size=latent_representation_size)
model.to(device)

#train_bVAE(model, train_loader, val_loader, beta=4, nb_epochs=600, device=device, saving_path='models/vae2D_mse_b4.pth')
model.load_state_dict(torch.load('models/vae2D_mse_b4.pth'))
model.eval()

In [ ]:
# TEST REGULAR VAE FOR RANDOM INDEX

test_dataset = Dataset2D(test_df)
test_image = test_dataset[3].to(device)
_, _, reconstructed_image, _ = model(test_image.unsqueeze(0))

plt.subplot(1, 2, 1)
plt.title('Original Image')
plt.imshow(test_image.squeeze(0).cpu().numpy())

plt.subplot(1, 2, 2)
plt.title('Reconstructed Image')
plt.imshow(reconstructed_image.squeeze(0).squeeze(0).cpu().detach().numpy())
plt.show()

In [ ]:
# TRAIN LONGITUDINAL VAE

from training_longitudinal_VAE2D import train
from leaspy import Leaspy, AlgorithmSettings, Data


model_longitudinal = VAE2D(latent_representation_size=4)
model_longitudinal.load_state_dict(torch.load('models/vae2D_mse_b4.pth'))
model_longitudinal.to(device)
model_longitudinal.freeze_conv()

estimator = Leaspy("linear", noise_model="gaussian_diagonal", source_dimension=latent_representation_size - 1)
algo_settings = AlgorithmSettings('mcmc_saem', n_iter=100, seed=45, noise_model="gaussian_diagonal")

#train(model_longitudinal, train_loader, val_loader, estimator, algo_settings, nb_epochs=50, beta=4, gamma = 0.4, batch_size=32, save_path = "models/longitudinal_vae2D.pth", device=device)

In [ ]:
# FIT DCM MODEL

model_longitudinal.load_state_dict(torch.load("models/longitudinal_vae2D.pth"))
model_longitudinal.eval()

estimator = Leaspy("linear", noise_model="gaussian_diagonal", source_dimension=latent_representation_size - 1)
algo_settings = AlgorithmSettings('mcmc_saem', n_iter=100, seed=45, noise_model="gaussian_diagonal")

from training_longitudinal_VAE2D import fit_longitudinal_estimator_on_nn
from utils import build_compatible_leaspy_dataframe

encodings_df = build_compatible_leaspy_dataframe(train_loader, model_longitudinal, device)
longitudinal_estimator, fitted_encodings = fit_longitudinal_estimator_on_nn(encodings_df, estimator, algo_settings)
longitudinal_estimator.save('models/leaspy_model')

In [ ]:
# EVALUATE ACCURACY ON EXTREME VALUES


extreme_dataset = DatasetLongitudinal2D(df_abnormal_subjects)
extreme_dataloader = DataLoader(extreme_dataset, batch_size=32, shuffle=False)
ids_dataset_extreme = ID_Dataset2D(df_abnormal_subjects)


from utils import evaluate_model_leaspy_on_extreme

acc, _ = evaluate_model_leaspy_on_extreme(model_longitudinal, df_abnormal_subjects, extreme_dataloader, ids_dataset_extreme, 'models/leaspy_model', device)
print(f"Accuracy on extreme values: {acc:.2f}%")